In [2]:
import os
import cv2
import pickle
import numpy as np

import sys

# sys.path.append("../arcface/")

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split


from tensorflow import keras
import tensorflow as tf

from arcface.dataset import build_dataset
from arcface.losses import ArcLoss
from arcface.network import ArcLayer, L2Normalization, hrnet_v2, resnet101
from arcface.training_supervisor import TrainingSupervisor

from tensorflow.keras.applications.efficientnet import EfficientNetB3

import IPython.display as display
from glob import glob
from tqdm import tqdm
from abc import ABC, abstractmethod

2022-05-18 19:55:10.910915: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
### For cpu only
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# TODO


1. hrnet (Done)
2. resnet ()
3. efficienthrnet
4. efficientnet


### TFRecord

In [4]:
def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def create_example(image, label):
    feature = {
        "image": image_feature(image),
        "label": bytes_feature(label),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

### Backbone Class make (ToDo Make into a separate module)############################
class RecordOperator(ABC):

    def __init__(self, filename):
        # Construct a reader if the user is trying to read the record file.
        self.dataset = None
        self._writer = None
        if tf.io.gfile.exists(filename):
            self.dataset = tf.data.TFRecordDataset(filename)
        else:
            # Construct a writer in case the user want to write something.
            self._writer = tf.io.TFRecordWriter(filename)

        # Set the feature description. This should be provided before trying to
        # parse the record file.
        self.set_feature_description()

    @abstractmethod
    def make_example(self):
        """Returns a tf.train.example from values to be saved."""
        pass

    def write_example(self, tf_example):
        """Create TFRecord example from a data sample."""
        if self._writer is None:
            raise IOError("Record file already exists.")
        else:
            self._writer.write(tf_example.SerializeToString())

    @abstractmethod
    def set_feature_description(self):
        """Set the feature_description to parse TFRecord file."""
        pass

    def parse_dataset(self):
        # Create a dictionary describing the features. This dict should be
        # consistent with the one used while generating the record file.
        if self.dataset is None:
            raise IOError("Dataset file not found.")

        def _parse_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            return tf.io.parse_single_example(example_proto, self.feature_description)

        parsed_dataset = self.dataset.map(_parse_function)
        return parsed_dataset
#########################################################################################

class ImageDataset(RecordOperator):
    """Construct ImageDataset tfrecord files."""

    def make_example(self, image, label):
        """Construct an tf.Example with image data and label.
        Args:
            image_string: encoded image, NOT as numpy array.
            label: the label.
        Returns:
            a tf.Example.
        """
        
        image_string = tf.image.decode_image(image)
        image_shape = image_string.shape
        

        # After getting all the features, time to generate a TensorFlow example.
        feature = {
            'image/height': int64_feature(image_shape[0]),
            'image/width': int64_feature(image_shape[1]),
            'image/depth': int64_feature(image_shape[2]),
            'image/encoded': image_feature(image_string),
            'label': int64_feature(label),
        }

        tf_example = tf.train.Example(
            features=tf.train.Features(feature=feature))

        return tf_example
    
    def set_feature_description(self):
        self.feature_description = {
            "image/height": tf.io.FixedLenFeature([], tf.int64),
            "image/width": tf.io.FixedLenFeature([], tf.int64),
            "image/depth": tf.io.FixedLenFeature([], tf.int64),
            "image/encoded": tf.io.VarLenFeature(tf.float32),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
#         example = tf.io.parse_single_example(example, feature_description)
#         example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
#         return example


def create_tfrecord(path = 'datasets/sorted_palmvein_roi/',  types=[".bmp"], tf_record='datasets/train.record'):
    
    converter = ImageDataset(tf_record)
    samples =[]
    [samples.extend(glob(path+"*/*."+typ)) for typ in types]
    total_samples_num = len(samples)
    ids = set()
    print("Total records: {}".format(total_samples_num))
    
    for i, image_path in tqdm(enumerate(samples)):
        image = tf.io.read_file(image_path)
        ids.add(image_path.split('/')[-2])
        label = len(ids)
        tf_example = converter.make_example(image, label)
        # Write the example to file.
        converter.write_example(tf_example)
        
    print("All done. Record file is:\n{}".format(tf_record))


In [5]:
name='merge'
create_tfrecord(path= 'datasets/processed_merge/', types=['png','jpg','jepg','png', 'bmp'], tf_record='datasets/'+name+'_processed.record')

Total records: 30902


30902it [00:15, 2044.87it/s]

All done. Record file is:
datasets/merge_processed.record


# Training

In [5]:
def train(base_model, name = "hrnetv2", train_files = "datasets/train.record", test_files = None, val_files = None, input_shape = (128, 128, 3),
          num_ids = 600, num_examples = 12000, training_dir = os.getcwd(),
          frequency = 1000, softmax = False, adam_alpha=0.001, adam_epsilon=0.001, batch_size = 8, export_only = False,
          override = False, epochs = 50, restore_weights_only=False):

    '''
    # Deep neural network training is complicated. The first thing is making
    # sure you have everything ready for training, like datasets, checkpoints,
    # logs, etc. Modify these paths to suit your needs.

    name:str = # What is the model's name?
    
    train_files:str = # Where are the training files?

    test_files:str = # Where are the testing files?

    val_files:str = # Where are the validation files? Set `None` if no files available.

    input_shape:tuple(int) = # What is the shape of the input image?

    embedding_size:int = # What is the size of the embeddings that represent the faces?

    num_ids:int = # How many identities do you have in the training dataset?

    num_examples:int = # How many examples do you have in the training dataset?

    # That should be sufficient for training. However if you want more
    # customization, please keep going.

    training_dir:str = # Where is the training direcotory for checkpoints and logs?

    regularizer = # Any weight regularization?

    frequency:int = # How often do you want to log and save the model, in steps?

    # All sets. Now it's time to build the model. There are two steps in ArcFace
    # training: 1, training with softmax loss; 2, training with arcloss. This
    # means not only different loss functions but also fragmented models.

    base_model:model = # First model is base model which outputs the face embeddings.
    '''
    
    # Where is the exported model going to be saved?
    export_dir = os.path.join(training_dir, 'exported', name)
    
    # Then build the second model for training.
    if softmax:
        print("Building training model with softmax loss...")
        model = keras.Sequential([keras.Input(input_shape),
                                  base_model,
                                  keras.layers.Dense(num_ids,
                                                     kernel_regularizer=regularizer),
                                  keras.layers.Softmax()],
                                 name="training_model")
        loss_fun = keras.losses.CategoricalCrossentropy()
    else:
        print("Building training model with ARC loss...")
        model = keras.Sequential([keras.Input(input_shape),
                                  base_model,
                                  L2Normalization(),
                                  ArcLayer(num_ids, regularizer)],
                                 name="training_model")
        loss_fun = ArcLoss()

    # Summary the model to find any thing suspicious at early stage.
    model.summary()

    # Construct an optimizer. This optimizer is different from the official
    # implementation which use SGD with momentum.
    optimizer = keras.optimizers.Adam(adam_alpha, amsgrad=True, epsilon=adam_epsilon)

    # Construct training datasets.
    dataset_train = build_dataset(train_files,
                                  batch_size=batch_size,
                                  one_hot_depth=num_ids,
                                  training=True,
                                  buffer_size=4096)

    # Construct dataset for validation. The loss value from this dataset can be
    # used to decide which checkpoint should be preserved.
    if val_files:
        dataset_val = build_dataset(val_files,
                                    batch_size=batch_size,
                                    one_hot_depth=num_ids,
                                    training=False,
                                    buffer_size=4096)

    # The training adventure is long and full of traps. A training supervisor
    # can help us to ease the pain.
    supervisor = TrainingSupervisor(model,
                                    optimizer,
                                    loss_fun,
                                    dataset_train,
                                    training_dir,
                                    frequency,
                                    "categorical_accuracy",
                                    'max',
                                    name)

    # If training accomplished, save the base model for inference.
    if export_only:
        print("The best model will be exported.")
        supervisor.restore(restore_weights_only, True)
        supervisor.export(base_model, export_dir)
        quit()

    # Restore the latest model if checkpoints are available.
    supervisor.restore(restore_weights_only)

    # Sometimes the training process might go wrong and we would like to resume
    # training from manually selected checkpoint. In this case some training
    # objects should be overridden before training started.
    if override:
        supervisor.override(0, 1)
        print("Training process overridden by user.")

    # Now it is safe to start training.
    supervisor.train(epochs, num_examples // batch_size)

    # Export the model after training.
    supervisor.export(base_model, export_dir)


In [6]:
# First model is bexported/e model which outputs the face embeddings.
name='merge'

input_shape = (128, 128, 3)
embedding_size = 512
num_ids = 1144
num_examples = 30942
batch_size = 64
epochs = 12

dataset_path = 'datasets/processed_merge'   
checkpoint_dir = 'outputs/exported/'+ name +'_soft/'

regularizer = keras.regularizers.L2(5e-4)
base_model = resnet101(input_shape=input_shape, output_size=embedding_size,
                           trainable=True,
                           training=True,
                           kernel_regularizer=regularizer,
                           name="embedding_model")

train(base_model, name = name+"_soft", train_files = "datasets/"+name+"_processed.record", test_files = None, val_files = None, input_shape = input_shape,
          num_ids = num_ids, num_examples = num_examples, training_dir = 'outputs/',
          frequency = 1000, softmax = True, adam_alpha=0.001, adam_epsilon=0.001, batch_size = batch_size, export_only = False,
          override = False, epochs = epochs, restore_weights_only=True)

2022-05-18 19:55:31.412812: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-18 19:55:31.413815: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-18 19:55:31.440095: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-18 19:55:31.440148: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Taavoos
2022-05-18 19:55:31.440159: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Taavoos
2022-05-18 19:55:31.440275: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-05-18 19:55:31.440310: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-05-18 19:55:31.440322: I tensorflow/stream_executor/cuda

Building training model with softmax loss...


KeyboardInterrupt: 

# Load softmax model and train with softmax = False

In [7]:
name='merge'

input_shape = (128, 128, 3)
embedding_size = 512
num_ids = 1144
num_examples = 30942
batch_size = 64
epochs = 12

dataset_path = 'datasets/processed_merge'   
checkpoint_dir = 'outputs/exported/'+ name +'_soft/'

regularizer = keras.regularizers.L2(5e-4)
base_model = keras.models.load_model(checkpoint_dir).expect_partial()


train(base_model, name = name+"_arc", train_files = "datasets/"+name+"_processed.record", test_files = None, val_files = None, input_shape = input_shape,
          num_ids = num_ids, num_examples = num_examples, training_dir = 'outputs/',
          frequency = 1000, softmax = False, adam_alpha=0.001, adam_epsilon=0.001, batch_size = batch_size, export_only = False,
          override = False, epochs = epochs, restore_weights_only=True)


Building training model with ARC loss...
Model: "training_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_model (Functional) (None, 512)               43685888  
_________________________________________________________________
l2_normalization (L2Normaliz (None, 512)               0         
_________________________________________________________________
arc_layer (ArcLayer)         (None, 1144)              585728    
Total params: 44,271,616
Trainable params: 44,168,832
Non-trainable params: 102,784
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to t

100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-| 482/483 [01:22<00:00,  6.71it/s, loss=3.53, accuracy=0.798]

Training accuracy: 0.7984, mean loss: 17.26
Monitor value improved from 0.0000 to 0.7984.
Best model found and saved: outputs/model_scout/merge_arc/ckpt-1


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-| 482/483 [01:25<00:00,  5.61it/s, loss=3.53, accuracy=0.798]


Checkpoint saved at global step: 482, to file: outputs/checkpoints/merge_arc/ckpt-2

Epoch 2/12


...

100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-| 482/483 [01:38<00:00,  4.83it/s, loss=0.34, accuracy=0.968]

Training accuracy: 0.9685, mean loss: 3.27
Monitor value improved from 0.9667 to 0.9685.
Best model found and saved: outputs/model_scout/merge_arc/ckpt-33


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-| 482/483 [01:41<00:00,  4.76it/s, loss=0.34, accuracy=0.968]

Checkpoint saved at global step: 5784, to file: outputs/checkpoints/merge_arc/ckpt-34
Training accomplished at epoch 12
Saving model to outputs/exported/merge_arc ...


INFO:tensorflow:Assets written to: outputs/exported/merge_arc/assets
Model saved at: outputs/exported/merge_arc


# Classifier

In [9]:
def save_pkl(pkl, path = 'model.pkl'):
    with open(path, 'wb') as f:
        pickle.dump(pkl, f)
    print("saved pkl file at:",path)

def load_pkl(path='model.pkl'):
    with open(path, 'rb') as f:
        pkl = pickle.load(f)
    return pkl

def findCosineDistance(vector1, vector2):
    """
    Calculate cosine distance between two vector
    """
    vec1 = vector1.flatten()
    vec2 = vector2.flatten()

    a = np.dot(vec1.T, vec2)
    b = np.dot(vec1.T, vec1)
    c = np.dot(vec2.T, vec2)
    return 1 - (a/(np.sqrt(b)*np.sqrt(c)))

def cosine_similarity(test_vec, source_vecs):
    """
    Verify the similarity of one vector to group vectors of one class
    """
    cos_dist = 0
    for source_vec in source_vecs:
        cos_dist += findCosineDistance(test_vec, source_vec)
    return cos_dist/len(source_vecs)

def make_embeddings(dataset_path='datasets/sorted_palmvein_roi/', output_path='outputs/', model_dir='outputs/exported/arcface', types='bmp'):
    # Grab the paths to the input images in our dataset
    print("[INFO] quantifying palms...")
    imagePaths =[]
    [imagePaths.extend(glob(dataset_path+"*/*."+typ)) for typ in types]
#     imagePaths.extend(glob(dataset_path+'/*/*.jpg'))

    # Initialize model
    embedding_model = keras.models.load_model(model_dir).expect_partial()
       
    # Initialize our lists of extracted facial embeddings and corresponding people names
    knownEmbeddings = []
    knownNames = []

    # Initialize the total number of faces processed
    total = 0

    # Loop over the imagePaths
    for (i, imagePath) in tqdm(enumerate(imagePaths)):
        # extract the person name from the image path
        name = imagePath.split(os.path.sep)[-2]
#         print(imagePath)

         # load the image
        img = cv2.imread(imagePath).reshape(-1,128,128,3)
        palms_embedding = embedding_model.predict(img)[0]
        # add the name of the person + corresponding face
        # embedding to their respective list
        knownNames.append(name)
        knownEmbeddings.append(palms_embedding)
        total += 1
        
    print(total, " palms embedded")
    print(f"number of classes: {len(set(knownNames))}")

    # save to output
    data = {"embeddings": knownEmbeddings, "names": knownNames}
    save_pkl(pkl=data, path=output_path+'db.pkl')
    
def make_model(embeddings_path='outputs/db.pkl', output_path='outputs/'):
    # Load the face embeddings
    data = load_pkl(embeddings_path)
    num_classes = len(np.unique(data["names"]))
    y = np.array(data["names"])
    X = np.array(data["embeddings"])
    
    
    # Initialize Softmax training model arguments
    input_shape = X.shape[1]
    cv = KFold(n_splits=5, random_state=1, shuffle=True)

    model =  MLPClassifier(hidden_layer_sizes=(input_shape, 640, 112, 640, num_classes), activation='tanh',max_iter=100, batch_size='auto', learning_rate='adaptive',
                           validation_fraction=0.0, solver='adam', early_stopping=False ,verbose=0,random_state=1)

    for train_idx, valid_idx in cv.split(X):
        model.fit(X[train_idx], y[train_idx],)
        print(model.score(X[valid_idx], y[valid_idx]), end='\t')
    
    save_pkl(pkl=model, path=output_path+'model.pkl')
    
    return model

In [10]:
make_embeddings(dataset_path='datasets/processed_merge/', output_path='outputs/exported/'+name+'_arc/', model_dir='outputs/exported/'+name+'_arc/',types=['png','jpg','jepg','png', 'bmp'])

[INFO] quantifying palms...


30902it [33:19, 15.46it/s]


30902  palms embedded
{1142}
saved pkl file at: outputs/exported/merge_arc/db.pkl


In [11]:
make_model(embeddings_path='outputs/exported/'+name+'_arc/db.pkl', output_path='outputs/exported/'+name+'_arc/')

0.9755702960685974	0.9676427762497978	0.9728155339805825	0.9697411003236246	0.9694174757281553	saved pkl file at: outputs/exported/merge_arc/model.pkl


MLPClassifier(activation='tanh', hidden_layer_sizes=(512, 640, 112, 640, 1142),
              learning_rate='adaptive', max_iter=10000, random_state=1,
              validation_fraction=0.0, verbose=0)

In [10]:
embedding_model = keras.models.load_model('outputs/exported/'+name+'_arc/')
model = load_pkl('outputs/exported/'+name+'_arc/model.pkl')
samples = glob('datasets/processed_merge/*/*.jpg')

In [13]:
img_paths = samples[0:20]
imgs = [cv2.imread(img).reshape(-1, 128, 128, 3) for img in img_paths]
embedding = [embedding_model.predict(img)[0] for img in imgs]

[print(np.format_float_positional((cosine_similarity(i, embedding[:10])), precision=3), end='\t') for i in embedding]
print()
[print(i, end='\t') for i in model.predict(embedding)]
print()
[print(img.split("/")[-2], end='\t') for img in img_paths] 
print()

0.583	0.456	0.488	0.505	0.483	0.499	0.549	0.480	0.48	0.575	0.815	0.704	0.680	0.775	0.830	0.731	0.787	0.812	0.8	0.783	
milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	
milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	


In [ ]:
embedding_model.save(